In [1]:
import myfm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from myfm.utils.benchmark_data import MovieLens100kDataManager
from sklearn.preprocessing import MultiLabelBinarizer, OneHotEncoder
import scipy.sparse as sps
from itertools import combinations
import numpy as np

In [2]:
path = 'E:\\recommendation-system\\CONTROL\\movies_metadata.csv'

In [3]:
df = pd.read_csv(path)

In [45]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [9]:
df

,adult,belongs_to_collection,budget,genres,homepage,movie_id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30/10/1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15/12/1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22/12/1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22/12/1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10/2/1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45459,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,17/11/2011,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45460,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,1/8/2003,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45461,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,21/10/1917,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [47]:
# change columns of pandas to true datatype
def change_to_true_dtype(data: pd.DataFrame):
    for i in data.columns:
        try:
            data[i] = pd.to_numeric(data[i])
        except ValueError:
            data[i] = data[i].astype('object')
    return data

# sort value of array
def sort_array(array):
    if array == 'nan' or array == 'NaN' or pd.isna(array):
        return 'NaN'
    new_array = [i['name'] for i in eval(array)]
    new_array.sort()
    return new_array

# check if value of column is dict
def check_value_is_dict(data: pd.DataFrame, column: str):
    for i in range(data[column].values.shape[0]):
        try:
            if type(eval(data[column][i])) == dict:
                return True
        except:
            return False
    return False

# check if value of column is list
def check_value_is_list(data: pd.DataFrame, column: str):
    for i in range(data[column].values.shape[0]):
        try:
            if type(eval(data[column][i])) == list:
                return True
        except:
            return False
    return False

# prepare data for predict
def prepare(data: pd.DataFrame):
    data = change_to_true_dtype(data)
    for i in data.columns:
        if 'id' in i:
            data.drop(columns=[i], axis=1, inplace=True)
            continue
        if 'id' not in i:
            if 'date' in i:
                data[i] = pd.to_datetime(data[i])
                # data[i] = [
                #     str(x) for x in data[i].dt.year.fillna('NaN')]
                data[i]=data[i].fillna('NaN')
                continue
            if pd.api.types.is_numeric_dtype(data[i].dtypes) and 'rating' not in i and data[i].dtypes not in [np.dtype('bool')]:
                data[i] = data[i].fillna(0)
                # data[i] = data[i] // 5 * 5
                continue
            if check_value_is_dict(data, i):
                data[i] = data[i].fillna('NaN')
                data[i] = data[i].apply(lambda x: 'NaN' if (
                    x=='nan' or x=='NaN' ) else eval(str(x))['name'])
                continue
            if check_value_is_list(data, i):
                data[i] = data[i].fillna('NaN')
                data[i] = data[i].apply(lambda x: sort_array(x))
                data[i] = data[i].apply(lambda x: 'NaN' if (
                    x=='nan' or x=='NaN' ) else '|'.join(
                    [i.replace(" ", "") for i in x]))
                continue
    return data.copy()

In [48]:
df_copy = df.copy()
df_copy = change_to_true_dtype(df_copy)
df_copy.dtypes

adult                       bool
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
movie_id                   int64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                    float64
vote_average             float64
vote_count               float64
dtype: object

In [49]:
df_copy = prepare(df_copy)
df_copy

c:\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '30/10/1995' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/12/1995' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '22/12/1995' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/11/1995' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent pars

,adult,belongs_to_collection,budget,genres,homepage,original_language,original_title,overview,popularity,poster_path,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,False,Toy Story Collection,30000000,Animation|Comedy|Family,http://toystory.disney.com/toy-story,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,UnitedStatesofAmerica,1995-10-30,373554033.0,81.0,English,Released,NaN,Toy Story,7.7,5415.0
1,False,NaN,65000000,Adventure|Family|Fantasy,NaN,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,UnitedStatesofAmerica,1995-12-15,262797249.0,104.0,English|Français,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,False,Grumpy Old Men Collection,0,Comedy|Romance,NaN,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,UnitedStatesofAmerica,1995-12-22,0.0,101.0,English,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0
3,False,NaN,16000000,Comedy|Drama|Romance,NaN,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,UnitedStatesofAmerica,1995-12-22,81452156.0,127.0,English,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0
4,False,Father of the Bride Collection,0,Comedy,NaN,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,UnitedStatesofAmerica,1995-10-02,76578911.0,106.0,English,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,NaN,0,Drama|Family,http://www.imdb.com/title/tt6209470/,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,...,Iran,NaT,0.0,90.0,فارسی,Released,Rising and falling between a man and woman,Subdue,4.0,1.0
45459,False,NaN,0,Drama,NaN,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,...,Philippines,2011-11-17,0.0,360.0,,Released,NaN,Century of Birthing,9.0,3.0
45460,False,NaN,0,Action|Drama|Thriller,NaN,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,...,UnitedStatesofAmerica,2003-01-08,0.0,90.0,English,Released,A deadly game of wits.,Betrayal,3.8,6.0
45461,False,NaN,0,,NaN,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,...,Russia,1917-10-21,0.0,87.0,,Released,NaN,Satan Triumphant,0.0,0.0


In [7]:
df2 = df.copy()
df2 = df2.drop(columns=["movie_id", "title", "original_title", "poster_path","imdb_id","spoken_languages","homepage","tagline"], axis=1)

In [51]:
def label_for_non_date_dtype(df_copy: pd.DataFrame):
    for column in df_copy.columns.values:
        print(column)
        if  (pd.api.types.is_numeric_dtype(df_copy[column].dtypes) == False or df_copy[column].dtypes in [np.dtype('bool')]) and 'date' not in column:
            uniqe_value = df_copy[column].unique()
            print(uniqe_value)
            print(len(uniqe_value))
            if len(uniqe_value) >= int(df_copy.values.shape[0]*0.8):
                df_copy.drop(columns=[column], inplace=True)
                continue
            new_value = {uniqe_value[i]: i for i in range(len(uniqe_value))} 
            # df_copy[[column]] = df_copy[[column]].replace(uniqe_value,[i for i in range(len(uniqe_value))])
            df_copy[[column]] = df_copy[[column]].replace(new_value)
            
    return df_copy.copy()

In [50]:
df_copy.values.shape[0]*0.8

36370.4

In [52]:
test = label_for_non_date_dtype(df_copy.copy())
test

adult
[False  True]
2
belongs_to_collection
['Toy Story Collection' 'NaN' 'Grumpy Old Men Collection' ...
 'Ducobu Collection' 'Mister Blot Collection' 'Red Lotus Collection']
1696
budget
genres
['Animation|Comedy|Family' 'Adventure|Family|Fantasy' 'Comedy|Romance' ...
 'Drama|Music|Romance|TVMovie'
 'Action|Adventure|Comedy|Family|Music|TVMovie'
 'Action|Animation|Comedy|Fantasy|Foreign|Horror|ScienceFiction']
1921
homepage
['http://toystory.disney.com/toy-story' nan
 'http://www.mgm.com/view/movie/757/Goldeneye/' ... 'http://ddpromo.ru/'
 'http://lmtr.fi/' 'http://www.imdb.com/title/tt6209470/']
7671
original_language
['en' 'fr' 'zh' 'it' 'fa' 'nl' 'de' 'cn' 'ar' 'es' 'ru' 'sv' 'ja' 'ko'
 'sr' 'bn' 'he' 'pt' 'wo' 'ro' 'hu' 'cy' 'vi' 'cs' 'da' 'no' 'nb' 'pl'
 'el' 'sh' 'xx' 'mk' 'bo' 'ca' 'fi' 'th' 'sk' 'bs' 'hi' 'tr' 'is' 'ps'
 'ab' 'eo' 'ka' 'mn' 'bm' 'zu' 'uk' 'af' 'la' 'et' 'ku' 'fy' 'lv' 'ta'
 'sl' 'tl' 'ur' 'rw' 'id' 'bg' 'mr' 'lt' 'kk' 'ms' 'sq' nan 'qu' 'te' 'am'
 'jv' 'tg' 'm

,adult,belongs_to_collection,budget,genres,homepage,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,0,0,30000000,0,0,0,21.946943,0,0,1995-10-30,373554033.0,81.0,0,0,0,7.7,5415.0
1,0,1,65000000,1,1,0,17.015539,1,0,1995-12-15,262797249.0,104.0,1,0,1,6.9,2413.0
2,0,2,0,2,1,0,11.712900,2,0,1995-12-22,0.0,101.0,0,0,2,6.5,92.0
3,0,1,16000000,3,1,0,3.859495,3,0,1995-12-22,81452156.0,127.0,0,0,3,6.1,34.0
4,0,3,0,4,1,0,8.387519,4,0,1995-10-02,76578911.0,106.0,0,0,4,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45458,0,1,0,44,7670,4,0.072051,49,17,NaT,0.0,90.0,18,0,20282,4.0,1.0
45459,0,1,0,21,1,57,0.178241,22650,101,2011-11-17,0.0,360.0,11,0,0,9.0,3.0
45460,0,1,0,37,1,0,0.903007,14593,0,2003-01-08,0.0,90.0,0,0,20283,3.8,6.0
45461,0,1,0,40,1,0,0.003503,22651,130,1917-10-21,0.0,87.0,11,0,0,0.0,0.0


In [32]:
test.release_date.dtypes

dtype('<M8[ns]')

In [41]:
def new_date(x,new_realease_day):
    for i in range(new_realease_day.index.shape[0]-1):
        if new_realease_day.index[i] < x <= new_realease_day.index[i+1]:
            return new_realease_day.index[i+1]

def label_for_date_dtype(df: pd.DataFrame):
    df['release_date'] = pd.to_datetime(df['release_date'])
    new_realease_day = df.resample('9M',on='release_date').count()

    indexDate = new_realease_day[ (new_realease_day['release_date'] == 0) ].index
    new_realease_day.drop(indexDate , inplace=True)
    
    for i in range(new_realease_day.index.shape[0]-1):
        df['release_date'] = df['release_date'].apply(lambda x: new_realease_day.index[i+1] if (new_realease_day.index[i] < x <= new_realease_day.index[i+1]) else x)
    unique_value = df['release_date'].unique()
    df['release_date'] = df['release_date'].replace(unique_value, [ i for i in range(len(unique_value))])
    return df

In [53]:
df_corr =  label_for_date_dtype(test.copy())

In [54]:
df_corr = df_corr.astype('float')
# result_corrcoef = np.corrcoef(df2.values,dtype=np.float32)
corrcoef = df_corr.corr()
# df3.values

In [55]:
corrcoef

,adult,belongs_to_collection,budget,genres,homepage,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
adult,1.000000,-0.003877,-0.003344,0.001558,0.003561,0.001816,-0.003391,0.005005,0.003996,0.001649,-0.002452,-0.008683,-0.003756,0.024705,0.016336,-0.012397,-0.002921
belongs_to_collection,-0.003877,1.000000,0.095170,0.117931,0.038093,0.040499,0.106360,0.056379,-0.010831,0.009919,0.126566,-0.010334,0.021586,-0.002904,0.025083,0.032243,0.120522
budget,-0.003344,0.095170,1.000000,0.026972,0.080832,-0.076110,0.449649,0.007667,-0.010391,0.072390,0.768776,0.135743,-0.001882,-0.021301,0.101687,0.073520,0.676643
genres,0.001558,0.117931,0.026972,1.000000,0.067391,0.019381,0.029719,0.079688,0.002535,0.018843,0.017940,0.002512,0.014132,0.004742,0.041705,0.009348,0.018220
homepage,0.003561,0.038093,0.080832,0.067391,1.000000,-0.000483,0.081983,0.218864,0.053598,0.293071,0.077815,0.010508,0.034307,0.021319,0.191819,0.037316,0.095450
original_language,0.001816,0.040499,-0.076110,0.019381,-0.000483,1.000000,-0.092035,0.137763,0.282346,0.068637,-0.056207,0.067694,0.461224,0.006865,-0.113297,0.061498,-0.067054
popularity,-0.003391,0.106360,0.449649,0.029719,0.081983,-0.092035,1.000000,0.040720,-0.021269,0.069833,0.506180,0.133894,-0.017551,-0.022587,0.115808,0.154449,0.559967
production_companies,0.005005,0.056379,0.007667,0.079688,0.218864,0.137763,0.040720,1.000000,0.208387,0.267641,0.001112,0.024279,0.125795,0.009777,0.211205,0.023961,0.025105
production_countries,0.003996,-0.010831,-0.010391,0.002535,0.053598,0.282346,-0.021269,0.208387,1.000000,0.101272,-0.022774,0.039045,0.318011,0.002191,-0.026927,0.037429,-0.016824
release_date,0.001649,0.009919,0.072390,0.018843,0.293071,0.068637,0.069833,0.267641,0.101272,1.000000,0.044334,-0.041160,0.100810,0.033977,0.097254,0.018350,0.064354


In [56]:
for i in corrcoef.columns.values:
    print("column ",i)
    print(corrcoef.sort_values(by=[i],ascending=False)[i])
    print("----------------------------")

column  adult
adult                    1.000000
status                   0.024705
tagline                  0.016336
production_companies     0.005005
production_countries     0.003996
homepage                 0.003561
original_language        0.001816
release_date             0.001649
genres                   0.001558
revenue                 -0.002452
vote_count              -0.002921
budget                  -0.003344
popularity              -0.003391
spoken_languages        -0.003756
belongs_to_collection   -0.003877
runtime                 -0.008683
vote_average            -0.012397
Name: adult, dtype: float64
----------------------------
column  belongs_to_collection
belongs_to_collection    1.000000
revenue                  0.126566
vote_count               0.120522
genres                   0.117931
popularity               0.106360
budget                   0.095170
production_companies     0.056379
original_language        0.040499
homepage                 0.038093
vote_average   